In [662]:
import pandas as pd
from datetime import datetime
import numpy as np
from datetime import timedelta, date
import matplotlib.dates as mdates
from scipy import stats
import datetime as dt
import psycopg2

connection = psycopg2.connect(user = "postgres", password = "admin", host = "127.0.0.1",port = "5432", database = "postgres")

def close_conn():
    if(connection):
        cursor.close()
        connection.close()
        
def get_data(stock,date='2018-01-01',date2='2019-01-01'):
    data = pd.read_sql_query("SELECT * FROM finance.stock_price where stock = '"+stock+"' and date between '"+date+"' and '"+date2+"' order by date asc",con=connection)
    return data

def get_stocks():
    data = pd.read_sql_query("SELECT a.stock, cap, vol, beta from "+
                            "( "+
                            "SELECT avg(marketcap) as cap, avg(beta) as beta, stock "+
                            "  FROM finance.stock_valuation_stats  where date > '2017-01-01' group by stock "+
                            ") a, " +
                            "( "+
                            "SELECT avg(vol) as vol, stock "+
                            "FROM finance.stock_price where date > '2017-01-01' group by stock "+
                            ") b "+
                            "  WHERE a.stock = b.stock "+
                            "    and cap is not null and beta >0.8 "+
                            "  order by cap desc, vol desc, beta desc ",con=connection)
    return data

def get_forex():
    data = pd.read_sql_query("SELECT symbol as stock FROM finance.stock where quote_type='CURRENCY' and symbol <>'ETHUSD=X' and symbol <> 'BTCUSD=X'" ,con=connection)
    return data

def get_sg_stocks():
    data = pd.read_sql_query("SELECT symbol as stock FROM finance.stock where country = 'Singapore'" ,con=connection)
    return data



In [806]:
class Strategy:
    def get_action(self, data, i, shares_can_buy, shares_can_sell, cash, target, risk_percent, risk_atr_multiplier):
        pass
    
    def set_stoploss(self, data, i, atr_period = 14, atr_multiplier = 3, early_stop_profit = 0.02, stop_early_times=1, danger_stop_atr=5):
        data.loc[i+1:,("stoploss")] = ATR_STOPLOSS(close = np.array(data["close"][i-atr_period:]), 
                                                         high = np.array(data["high"][i-atr_period:]),
                                                         low = np.array(data["low"][i-atr_period:]), 
                                     times=atr_multiplier, period=atr_period, early_stop_profit=early_stop_profit,stop_early_times=stop_early_times)[0][atr_period+1:]
        data.loc[i+1:,("stoploss_fix")] = ATR_STOPLOSS(close = np.array(data["close"][i-atr_period:]), 
                                                         high = np.array(data["high"][i-atr_period:]),
                                                         low = np.array(data["low"][i-atr_period:]), 
                                     times=atr_multiplier, period=atr_period, early_stop_profit=early_stop_profit,stop_early_times=atr_multiplier)[0][atr_period+1:]
        data.loc[i+1:,("stoploss_danger")] = ATR_STOPLOSS(close = np.array(data["close"][i-atr_period:]), 
                                                         high = np.array(data["high"][i-atr_period:]),
                                                         low = np.array(data["low"][i-atr_period:]), 
                                     times=danger_stop_atr, period=atr_period, early_stop_profit=early_stop_profit,stop_early_times=danger_stop_atr)[0][atr_period+1:]
        
        return data

    def get_col_to_plot(self):
        pass

In [848]:
def ATR_STOPLOSS(close, high, low, times=3, period=14, early_stop_profit=0.02, stop_early_times=3, repaint = False, median=False, average_high_low=False):
    starting_close = close[period]
    from talib import ATR
    if(average_high_low == False):
        if(median==False):
            atr= ATR(high,low, close ,timeperiod=period)
        else:
            atr=MATR(high,low, close ,timeperiod=period)
    else:
        atr=CLOSE_ATR(high,low, close ,timeperiod=period)
    first = 1
    import math
    atr_trailing_stoploss = np.full(len(close)+1, np.nan)
    for i in range(0,len(close)):
        if(i>0 and math.isnan(atr[i])==False and repaint==True):
            if((close[i-1]<atr_trailing_stoploss[i-1] and close[i]>atr_trailing_stoploss[i])
              or (close[i-1]>atr_trailing_stoploss[i-1] and close[i]<atr_trailing_stoploss[i])):
                starting_close = close[i]
        multiplier = times
        if(i>0 and math.isnan(atr[i])==False):
            if(close[i-1]>atr_trailing_stoploss[i] and close[i]<atr_trailing_stoploss[i]):
                loss = atr[i]*multiplier
                atr_trailing_stoploss[i+1]=close[i]+loss  
                continue
        if(i>0 and math.isnan(atr[i])==False):
            if(close[i-1]<atr_trailing_stoploss[i] and close[i]>atr_trailing_stoploss[i]):
                loss = atr[i]*multiplier
                atr_trailing_stoploss[i+1]=close[i]-loss  
                continue
        if(math.isnan(atr[i])==False):
            loss = atr[i]*multiplier
            atr_trailing_stoploss[i+1]=close[i]-loss
            if(close[i]<atr_trailing_stoploss[i]):
                if(atr_trailing_stoploss[i]>close[i]+loss):
                    '''
                    ratio_to_target = ((close[i]-starting_close)/(starting_close))/early_stop_profit
                    if(((close[i]-starting_close)/(starting_close))<-early_stop_profit):
                        loss = atr[i]*stop_early_times
                    else:
                        if(ratio_to_target<0):
                            loss = atr[i]*(times-((times - stop_early_times)*(-ratio_to_target)))
                        else:
                            loss = atr[i]*multiplier
                    '''
                    atr_trailing_stoploss[i+1]=close[i]+loss
                else:
                    if(math.isnan(atr_trailing_stoploss[i])==False):
                        atr_trailing_stoploss[i+1]=atr_trailing_stoploss[i]
            else:
                if(atr_trailing_stoploss[i]<close[i]-loss):
                    ratio_to_target = ((close[i]-starting_close)/(starting_close))/early_stop_profit
                    if(((close[i]-starting_close)/(starting_close))>early_stop_profit):
                        loss = atr[i]*stop_early_times
                    else:
                        if(ratio_to_target>0):
                            loss = atr[i]*(times-((times - stop_early_times)*ratio_to_target))
                        else:
                            loss = atr[i]*multiplier
                    
                    atr_trailing_stoploss[i+1]=close[i]-loss
                else:
                    if(math.isnan(atr_trailing_stoploss[i])==False):
                        atr_trailing_stoploss[i+1]=atr_trailing_stoploss[i]
    return atr_trailing_stoploss[:-1], atr_trailing_stoploss[-1:]


In [855]:
def plot_chart(stocks_prices, result, stock, strategy):
    ######Plot
    ######
    growth, longs, shorts,  win_trade_count, loss_trade_count, win_percent, loss_percent, final_cash=result
    print("profit: "+str(round(growth,2))+"%"+"\t"+"asset: "+str(final_cash))
    print("\t"+"win count: "+str(win_trade_count[stock])+"\t"+"loss count: "+str(loss_trade_count[stock]))
    print("\t"+"win percent: "+str(win_percent[stock])+"\t"+"loss percent: "+str(loss_percent[stock]))
    import plotly.graph_objs as go 
    from datetime import datetime
    from ipywidgets import interact, interactive, fixed, interact_manual
    import ipywidgets as widgets
    from plotly.subplots import make_subplots
    # Make sure dates are in ascending order
    # We need this for slicing in the callback below
    data = stocks_prices[stock]
    df = data.set_index('date')
    fig = go.FigureWidget(make_subplots(rows=4, cols=1))
    
    
    
    fig.add_trace(
        go.Scattergl(x=list(df.index), y=list(df['close']),
        name = "close"),
        row=1, col=1
    )
    fig.add_trace(
        go.Scattergl(x=list(df.index), y=list(df['stoploss']), line = dict(color='firebrick', width=2, dash='dash'),
        name = "stoploss"),
        row=1, col=1
    )
    fig.add_trace(
        go.Scattergl(x=list(df.index), y=list(df['stoploss_fix']), line = dict(color='firebrick', width=2, dash='dash'),
        name = "stoploss_fix"),
        row=1, col=1
    )
    fig.add_trace(
        go.Scattergl(x=list(df.index), y=list(df['stoploss_danger']), line = dict(color='firebrick', width=2, dash='dash'),
        name = "stoploss_fix"),
        row=1, col=1
    )
    fig.add_trace(
        go.Scattergl(x=[i for i in shorts[stock]], y=[shorts[stock][i] for i in shorts[stock]],
        name = "Short", mode="markers", marker=dict(color="red", size=5)),
        row=1, col=1
    )
    fig.add_trace(
        go.Scattergl(x=[i for i in longs[stock]], y=[longs[stock][i] for i in longs[stock]],
        name = "Long", mode="markers", marker=dict(color="green", size=5)),
        row=1, col=1
    )
    plot1,plot2,plot3 = strategy.get_col_to_plot()
    
    for p in plot1:
        fig.add_trace(
            go.Scattergl(x=list(df.index), y=list(df[p]),
            name = p),
            row=1, col=1
        )
    for p in plot2:
        fig.add_trace(
            go.Scattergl(x=list(df.index), y=list(df[p]),
            name = p),
            row=2, col=1
        )
    for p in plot3:
        fig.add_trace(
            go.Scattergl(x=list(df.index), y=list(df[p]),
            name = p),
            row=3, col=1
        )
    fig.layout.xaxis=dict(
            anchor='x',
            rangeselector=dict(
                buttons=list([
                    dict(count=1,
                         label='1m',
                         step='month',
                         stepmode='backward'),
                    dict(count=6,
                         label='6m',
                         step='month',
                         stepmode='backward'),
                    dict(count=1,
                        label='YTD',
                        step='year',
                        stepmode='todate'),
                    dict(count=1,
                        label='1y',
                        step='year',
                        stepmode='backward'),
                    dict(step='all')
                ])
            ),
            type='date'
        )
    fig.layout.yaxis=dict(
            domain=[0.53, 1]
        )
    fig.layout.yaxis2=dict(
            domain=[0.26, 0.5]
        )
    fig.layout.yaxis3=dict(
            domain=[0, 0.24]
        )
    #fig.layout.yaxis2.range=[0,100]
    #fig.layout.yaxis2.tickvals=[0,20,80,100]
    fig['layout'].update(height=600, width=1000) 
    
    import time
    import datetime
    def zoom(layout, xrange):
        try:
            fig.layout.xaxis2.range = fig.layout.xaxis.range
            fig.layout.xaxis3.range = fig.layout.xaxis.range
            fig.layout.xaxis4.range = fig.layout.xaxis.range
            in_view = df.loc[fig.layout.xaxis.range[0]:fig.layout.xaxis.range[1]]
            padding = 0.1
            
            from_price = in_view.iloc[0].close
            to_price = in_view.iloc[-1].close
            growth = (to_price-from_price)/from_price*100.0
            
            max_val_y = in_view["close"].max()
            min_val_y =  in_view["close"].min()
            axis_diff = max_val_y-min_val_y
            fig.layout.yaxis.range = [min_val_y - (axis_diff*padding), max_val_y + (axis_diff*padding)]
            
            if(len(plot2)>0):
                max_val_y = in_view[plot2[0]].max()
                min_val_y =  in_view[plot2[0]].min()
                axis_diff = max_val_y-min_val_y
                fig.layout.yaxis2.range = [min_val_y - (axis_diff*padding), max_val_y + (axis_diff*padding)]
         
            if(len(plot3)>0):
                max_val_y = in_view[plot3[0]].max()
                min_val_y =  in_view[plot3[0]].min()
                axis_diff = max_val_y-min_val_y
                fig.layout.yaxis3.range = [min_val_y - (axis_diff*padding), max_val_y + (axis_diff*padding)]
         
            fig['layout'].update(title=symbol+" ("+str(round(growth,2))+"%)") 
        except Exception as e:
            None
    fig.layout.on_change(zoom, 'xaxis')
    return fig

In [875]:
def back_test(data, cash = 100000,transaction_fee = 0.01, assets={}, strategy=StrategyMACD(), min_transaction = 1000, target=0.02, risk_percent = 0.01, risk_atr_multiplier = 3):
    longs = {}
    shorts = {}
    init_cash = cash
    bought_at = 100000
    bought_at_list = []
    loss_trade_count = 0
    win_trade_count = 0
    win_percent = 0
    loss_percent = 0
    
    process_col(data,"atr_risk",14)
    for i,row in data.iterrows():
        shares_can_buy = int(cash/(row.close*(1+transaction_fee)))
        shares_can_sell=0
        if row.stock in assets:
            shares_can_sell = assets[row.stock]
        #Decision
        recommended_shares = -1
        shares_held = 0
        if(i>0):
            current_assets = cash
            if(data.iloc[i].stock in assets):
                shares_held = assets[data.iloc[i-1].stock]
                current_assets = shares_held*data.iloc[i-1].close + cash
            risk_atr = data["atr_risk_14"][i-1]
            last_close = data["close"][i-1]
            import math
            if(math.isnan(risk_atr)==False):
                risk_atr =  risk_atr * risk_atr_multiplier
                risk_amt = current_assets * risk_percent
                if(risk_atr!=0):
                    recommended_shares = int(risk_amt/risk_atr)
                    if(shares_can_buy>0):
                        if(shares_held+shares_can_buy>recommended_shares):
                            shares_can_buy = int(recommended_shares-shares_held)            
                else:
                    continue
            else:
                continue
        #if(recommended_shares<=0 or recommended_shares*row.close*(1+transaction_fee)<min_transaction):
        #    continue
        action , price_point, data = strategy.get_action(data, i, shares_can_buy, shares_can_sell, target, risk_percent, risk_atr_multiplier)
        #Execution
        if(action=="BUY"):
            if(shares_can_buy>0):
                cost = shares_can_buy*price_point*(1+transaction_fee)
                cash_left = cash-cost
                if row.stock not in assets:
                    assets[row.stock]=0
                assets[row.stock] = assets[row.stock] + shares_can_buy
                cash = cash_left
                longs[row.date] = price_point
                bought_at = price_point
                bought_at_list = bought_at_list + [price_point]
                #print("BUY : +" +str(shares_can_buy) +" @"+str(price_point))
        if(action=="SELL"):
            if(shares_can_sell>0):
                asset = shares_can_sell * price_point
                cash = cash + asset*(1-transaction_fee)
                assets.clear()
                shorts[row.date] = price_point
                #trade counter
                for trade in bought_at_list:
                    if(trade>price_point):
                        loss_trade_count = loss_trade_count +1
                        loss_percent = loss_percent + ((price_point-trade)/trade)
                    else:
                        win_trade_count = win_trade_count +1
                        win_percent = win_percent + ((price_point-trade)/trade)
                bought_at_list = []
                #print("SELL : -" +str(shares_can_sell) +" @"+str(price_point))
        if(action=="STOPPED"):
            if(shares_can_sell>0):
                asset = shares_can_sell * price_point
                cash = cash + asset*(1-transaction_fee)
                assets.clear()
                shorts[row.date] = price_point
                #trade counter
                for trade in bought_at_list:
                    if(trade>price_point):
                        loss_trade_count = loss_trade_count +1
                        loss_percent = loss_percent + ((price_point-trade)/trade)
                    else:
                        win_trade_count = win_trade_count +1
                        win_percent = win_percent + ((price_point-trade)/trade)
                bought_at_list = []
                #print("STOPPED : -" +str(shares_can_sell) +" @"+str(price_point))
    final_cash = cash
    if(data.iloc[-1].stock in assets):
        final_cash = assets[data.iloc[-1].stock]*data.iloc[-1].close + cash
    assets.clear()
    growth =((final_cash-init_cash)/init_cash)*100
    print("profit: "+str(round(growth,2))+"%"+"\t"+"asset: "+str(final_cash))
    print("\t"+"win count: "+str(win_trade_count)+"\t"+"loss count: "+str(loss_trade_count))
    print("\t"+"win percent: "+str(win_percent)+"\t"+"loss percent: "+str(loss_percent))
    return round(growth,2), longs, shorts,  win_trade_count, loss_trade_count


In [876]:
def SSL(high,low, close, timeperiod=7):
    from talib import EMA
    sma_high = EMA(high,timeperiod )
    sma_low = EMA(low,timeperiod )
    
    ssl_up = np.full(len(close), np.nan)
    ssl_down = np.full(len(close), np.nan)
    lv = np.full(len(close), np.nan)
    lv[timeperiod-0]=1
    for i in range(0,len(close)):
        if(i>=timeperiod):
            if(close[i]<sma_low[i]):
                lv[i] = -1
            else:
                if(close[i]>sma_high[i]):
                    lv[i] = 1
                else:
                    lv[i] = 0
            if(lv[i]==0):
                lv[i] = lv[i-1]
                
            if(lv[i]>0):
                ssl_up[i] = sma_high[i]
                ssl_down[i] = sma_low[i]
            else:
                ssl_up[i] = sma_low[i]
                ssl_down[i] = sma_high[i]
    return ssl_up, ssl_down
def RVI(high, low, close, open, timeperiod = 7):
    rvi = np.full(len(close), np.nan)
    signal = np.full(len(close), np.nan)
    numerator = np.full(len(close), np.nan)
    denominator = np.full(len(close), np.nan)
    for i in range(0,len(close)):
        if(i>=3):
            a = close[i] - open[i]
            b = close[i-1] - open[i-1] 
            c = close[i-2] - open[i-2] 
            d = close[i-3] - open[i-3] 
            e = high[i] - low[i]
            f = high[i-1]  - low[i-1] 
            g = high[i-2]  - low[i-2] 
            h = high[i-3]  - low[i-3]
            numerator[i]=(a+(2*b)+(2*c)+d)/6
            denominator[i]=(e+(2*f)+(2*g)+h)/6
        if(i>=(3+timeperiod)):
            sma_numerator = np.array(numerator)[i-(timeperiod-1):i+1].mean()
            sma_denominator = np.array(denominator)[i-(timeperiod-1):i+1].mean()
            rvi[i] = sma_numerator/sma_denominator
        
        if(i>=6+timeperiod):
            signal[i] = (rvi[i] + (2*rvi[i-1]) + (2*rvi[i-2]) +rvi[i-3])/6
    return rvi,signal

def VWAP(vol, high, low, close):
    cum_vol = vol.cumsum()
    cum_vol_price = (vol * (high + low + close ) /3).cumsum()
    return cum_vol_price / cum_vol

def MATR(high,low, close ,timeperiod=14):
    import statistics
    tr = high - low
    matr = np.full(len(close)+1, np.nan)
    for i in range(0,len(matr)):
        if(i>=timeperiod):
            matr[i]=statistics.median(np.array(tr[i-(timeperiod-1):i+1]))
    return matr

def CLOSE_ATR(high,low, close ,timeperiod=14):
    from talib import SMA
    sma_high = SMA(high,timeperiod )
    sma_low = SMA(low,timeperiod )
    import statistics
    tr = sma_high - sma_low
    matr = np.full(len(close)+1, np.nan)
    for i in range(0,len(matr)):
        if(i>=timeperiod):
            matr[i]=(np.array(tr[i-(timeperiod-1):i+1])).max()
    return matr

def Normalise(listss):
    min_i = min(listss)
    max_i = max(listss)
    if(min_i==max_i):
        return listss
    return [(x-min_i)/(max_i-min_i) for x in listss]
def SLOPE(data):
    #data = np.array(Normalise(data))
    #data = data * (len(data)-1)
    from talib import LINEARREG_ANGLE
    return list(LINEARREG_ANGLE(data,timeperiod = len(data)-1))[-1]
def PRICE_SLOPE(data, period = 14):
    df = data.tail(period)
    first = df.index.start
    last = df.index.stop
    x = df.index.values.reshape(-1, 1) 
    y= df.close.values.reshape(-1, 1) 
    from sklearn.linear_model import LinearRegression
    reg = LinearRegression().fit(x,y)
    first_predicted = reg.predict(np.array([[first]]))[0][0]
    last_predicted = reg.predict(np.array([[last-1]]))[0][0]
    gradient =(last_predicted-first_predicted)/(last-first)
    return gradient


def HEIKIN_ASHI(open_price, high_price, low_price, close_price):
    ha_close = (open_price+ high_price+ low_price+ close_price) / 4
    ha_open = np.full(len(open_price), np.nan)
    ha_high = np.full(len(open_price), np.nan)
    ha_low = np.full(len(open_price), np.nan)
    for i in range(len(open_price)):
        if i == 0:
            ha_open[0]= open_price[0]
        else:
            ha_open[i] = (ha_open[i-1] + ha_close[i-1]) / 2
        ha_high[i] = np.array([high_price[i], ha_open[i], ha_close[i]]).max()
        ha_low[i] = np.array([low_price[i], ha_open[i], ha_close[i]]).min()
    return ha_open, ha_high, ha_low, ha_close

    '''
    if(i>60):
            last_60days_trend_slope = PRICE_SLOPE(data[0:i], period = 60)
            last_60days_max_high = data[0:i].tail(60).high.max()
            last_60days_min_low = data[0:i].tail(60).low.min()
    '''

    '''
        if(i>30):
            last_30days_trend_slope = PRICE_SLOPE(data[0:i], period = 30)
            last_30days_max_high = data[0:i].tail(30).high.max()
            last_30days_min_low = data[0:i].tail(30).low.min()
    '''

    '''
        if(i>14):
            last_14days_trend_slope = PRICE_SLOPE(data[0:i], period = 14)
            last_14days_max_high = data[0:i].tail(14).high.max()
            last_14days_min_low = data[0:i].tail(14).low.min()
            price_perct_14days = (data["close"][i]-last_14days_min_low)/(last_14days_max_high-last_14days_min_low)
    '''

    '''
        max_rsi = data["rsi"][0:i].max()
        min_rsi = data["rsi"][0:i].min()
        rsi_threshold = (max_rsi-min_rsi)*0.02
    '''

def LINEAR_REGRESSION_BOUND(data_close, data_high, data_low, period = 14, skip = 0):
    linreg_upper = np.full(len(data_close), np.nan)
    linreg_lower = np.full(len(data_close), np.nan)
    for i in range(len(data_close)):
        if(i>(period-1+skip)):
            close = np.array(data_close[i-(period)-skip:i-skip])
            high = np.array(data_high[i-(period)-skip:i-skip])
            low = np.array(data_low[i-(period)-skip:i-skip])
            first = 0
            last = period
            x = np.array(list(range(first,last))).reshape(-1, 1) 
            y= np.array(close).reshape(-1, 1) 
            from sklearn.linear_model import LinearRegression
            reg = LinearRegression().fit(x,y)

            max_high1 = 0
            max_high2 = 0
            max_low1 = 0
            max_low2 = 0
            for index in range(len(close)):
                predicted_price = reg.predict(np.array([[index]]))[0][0]
                high_diff = high[index] - predicted_price
                if(max_high1<high_diff):
                    max_high2 = max_high1
                    max_high1 = high_diff
                else:
                    if(max_high2<high_diff):
                        max_high2 = high_diff

                low_diff =  predicted_price - low[index]
                if(max_low1<low_diff):
                    max_low2 = max_low1
                    max_low1 = low_diff
                else:
                    if(max_low2<low_diff):
                        max_low2 = low_diff
            predicted_close = reg.predict(np.array([[len(close)+skip]]))[0][0]
            linreg_upper[i] = predicted_close+((max_high1+max_high2)/2.0)
            linreg_lower[i] = predicted_close-((max_low1+max_low2)/2.0 )
    return linreg_upper, linreg_lower

def PAST_LOW_HIGH(data, period = 14):
    slop_angle = np.full(len(data), np.nan)
    past_high = np.full(len(data), np.nan)
    past_low = np.full(len(data), np.nan)
    for i in range(len(data)):
        if(i>=period):
            slop_angle[i] = SLOPE(data[i-period:i].close)
            past_high[i] = data[0:i-1].tail(period).high.max()
            past_low[i] = data[0:i-1].tail(period).low.min()
    return slop_angle, past_high, past_low
    
    
def STOCHARSI(rsi, timeperiod=14):
    stocha_rsi = np.full(len(rsi), np.nan)
    for i in range(len(rsi)):
        if(i>=14):
            curr_rsi = rsi[i]
            min_rsi = rsi[i-14:i].min()
            max_rsi = rsi[i-14:i].max()
            stocha_rsi[i]=((curr_rsi-min_rsi)/(max_rsi-min_rsi))*100
            if(stocha_rsi[i]>100):
                stocha_rsi[i]=100
            if(stocha_rsi[i]<0):
                stocha_rsi[i]=0
    return stocha_rsi

def KIJUNSEN(high,low, timeperiod=26):
    kijunsen = np.full(len(high), np.nan)
    for i in range(len(high)):
        if(i>=timeperiod):
            kijunsen[i]=(high[i-timeperiod:i].max()+low[i-timeperiod:i].min())/2
    return kijunsen
    

def HMA(close, timeperiod=14):
    import math
    from talib import WMA
    sqrt_period = math.sqrt(timeperiod)
    wma1 = (2*WMA(close, timeperiod = int(timeperiod/2)))-WMA(close, timeperiod = timeperiod)
    return WMA(wma1,timeperiod =int(sqrt_period))
    

In [877]:
def process_col(data, col="", *argv):
    params = '_'.join(str(x) for x in argv)
    if(col+"_"+params in data.columns):
        return
    
    if(col=="zero"):
        data['zero'] = np.full(len(data), 0)
       
    if(col=="atr_risk"):
        from talib import ATR
        data["atr_risk_"+params]= ATR(data['high'].values, data['low'].values, data['close'].values ,timeperiod=argv[0])
    
    if(col=="macd"):
        from talib import MACD
        data['macd_'+params], data['macd_signal_'+params], data['macd_hist_'+params] = MACD(data['close'], fastperiod=argv[0], slowperiod=argv[1], signalperiod=argv[2])

    if(col=="rsi"):
        from talib import RSI
        data['rsi_'+params] = RSI(data['close'].values, timeperiod=argv[0])

    if(col=="adx"):
        from talib import ADX
        data['adx_'+params] =  ADX(data['high'].values, data['low'].values, data['close'].values, timeperiod=argv[0])

    if(col=="kijunsen"):
        data['kijunsen_'+params] = KIJUNSEN(data['high'],data['low'], timeperiod=argv[0])
        
    if(col=="ema"):
        from talib import EMA
        print("@@@")
        data['ema_'+params] = EMA(data[argv[0]], timeperiod=argv[1])
        
    if(col=="sma"):
        from talib import SMA
        data['sma_'+params] = SMA(data[argv[0]], timeperiod=argv[1])
        
    if(col=="hma"):
        data['hma_'+params] = HMA(data[argv[0]], timeperiod=argv[1])
        
    if(col=="linearreg"):
        from talib import LINEARREG_ANGLE
        data['linearreg_'+params] = LINEARREG_ANGLE(data[argv[0]], timeperiod=argv[1])
        
    if(col=="linearreg"):
        from talib import LINEARREG_ANGLE
        data['linearreg_'+params] = LINEARREG_ANGLE(data[argv[0]], timeperiod=argv[1])
    
    if(col=="atr_stoploss"):
        from talib import LINEARREG_ANGLE
        data['atr_stoploss_'+params] = ATR_STOPLOSS(close = data.close.values, 
                                                   high = data.high.values, 
                                                   low = data.low.values, 
                                 times=argv[0], stop_early_times=argv[1], early_stop_profit=argv[2], period=argv[3], repaint=True)[0]

    if(col=="atr"):
        from talib import ATR
        data['atr_'+params] = ATR(data['high'].values, data['low'].values, data['close'].values ,timeperiod=argv[0])
    
    
    if(col=="ssl"):
        data["ssl_up_"+params],quotes["ssl_down_"+params]= SSL(data['high'].values, data['low'].values, data['close'].values ,timeperiod=argv[0])
    
    
    if(col=="ha"):
        data["ha_open"],data["ha_high"],data["ha_low"],data["ha_close"] = HEIKIN_ASHI(data['open'].values, data['high'].values, data['low'].values, data['close'].values)

def name_col(col="", *argv):
    params = '_'.join(str(x) for x in argv)
    return col+"_"+params

In [878]:
class StrategyMACD1(StrategyMACD):
    def __init__(self):
        self.macd_param_1 = 20
        self.macd_param_2 = 50
        self.macd_param_3 = 5
    
    
class StrategyMACD(Strategy):
    def __init__(self):
        self.macd_param_1 = 12
        self.macd_param_2 = 26
        self.macd_param_3 = 9
    
    def get_action(self, data, i, shares_can_buy, shares_can_sell, target, risk_percent, risk_atr_multiplier):
        
        self.macd = name_col("macd",self.macd_param_1,self.macd_param_2,self.macd_param_3)
        self.macd_signal = name_col("macd_signal",self.macd_param_1,self.macd_param_2,self.macd_param_3)
        self.macd_hist = name_col("macd_hist",self.macd_param_1,self.macd_param_2,self.macd_param_3)
        
        process_col(data,"macd",self.macd_param_1,self.macd_param_2,self.macd_param_3)
        
        action ="HOLD", 0 , data
        
        
        if(
            data[self.macd][i]>data[self.macd_signal][i]
            and data[self.macd][i-1]<data[self.macd_signal][i-1]
        ):
            data = self.set_stoploss(data, i , 14, risk_atr_multiplier, 
                early_stop_profit=target ,stop_early_times=1.5, danger_stop_atr=6)
            action = "BUY", data.close[i], data
            
            
        if(
            data[self.macd][i]<data[self.macd_signal][i]
            and data[self.macd][i-1]>data[self.macd_signal][i-1]
        ):
            action = "SELL", data.close[i], data

        '''
        if(shares_can_sell>0):
            if(i>0 
               and data["stoploss"][i]>data.low[i]
              ):
                action = "STOPPED", data["stoploss"][i], data
        
        '''
        return action
    
    def get_col_to_plot(self):
        plot1 = []
        plot2 = []
        plot3 = []
        
        plot2.append(self.macd)
        plot2.append(self.macd_signal)
        plot2.append(self.macd_hist)
        
        return plot1,plot2,plot3
    
    
#target=0.07, risk_percent = 0.01, risk_atr_multiplier =3
class Strategy3(Strategy):
    def __init__(self):
        self.kijunsen_param_1 = 12
        self.ema_close_1_param_1 = 'close'
        self.ema_close_1_param_2 = 7
        self.ema_close_2_param_1 = 'close'
        self.ema_close_2_param_2 = 20
        self.atr_param_1 = 14
        self.atr_stoploss_1_param_1 = 3
        self.atr_stoploss_1_param_2 = 3
        self.atr_stoploss_1_param_3 = 0.02
        self.atr_stoploss_1_param_4 = 14
        self.macd_param_1=12
        self.macd_param_2=26
        self.macd_param_3=9
        
    def get_action(self, data, i, shares_can_buy, shares_can_sell, target, risk_percent, risk_atr_multiplier):

        import random
        
        self.kijunsen = name_col("kijunsen",self.kijunsen_param_1)
        self.ema_close_1 = name_col("ema",self.ema_close_1_param_1,self.ema_close_1_param_2)
        self.ema_close_2 = name_col("ema",self.ema_close_2_param_1,self.ema_close_2_param_2)
        self.ha_open = "ha_open"
        self.ha_close = "ha_close"
        self.ha_high = "ha_high"
        self.ha_low ="ha_low"
        self.atr = name_col("atr",self.atr_param_1)
        self.macd = name_col("macd",self.macd_param_1,self.macd_param_2,self.macd_param_3)
        self.macd_signal = name_col("macd_signal",self.macd_param_1,self.macd_param_2,self.macd_param_3)
        self.macd_hist = name_col("macd_hist",self.macd_param_1,self.macd_param_2,self.macd_param_3)
        self.atr_stoploss_1 = name_col("atr_stoploss",self.atr_stoploss_1_param_1,
                              self.atr_stoploss_1_param_2,self.atr_stoploss_1_param_3,
                              self.atr_stoploss_1_param_4)
        
        
        process_col(data,"kijunsen",self.kijunsen_param_1)
        process_col(data,"ema",self.ema_close_1_param_1,self.ema_close_1_param_2)
        process_col(data,"ema",self.ema_close_2_param_1,self.ema_close_2_param_2)
        process_col(data,"ha")
        process_col(data,"atr",self.atr_param_1)
        process_col(data,"macd",self.macd_param_1,self.macd_param_2,self.macd_param_3)
        process_col(data,"atr_stoploss",self.atr_stoploss_1_param_1,
                              self.atr_stoploss_1_param_2,self.atr_stoploss_1_param_3,
                              self.atr_stoploss_1_param_4)
        
        action ="HOLD", 0, data
        
        if(i<100):
            return action
        ########################################################
        #Base Line 1
        baseline_signal = 0
        if(data["close"][i]-data[self.kijunsen][i]>0):
            for p in range(1,7):
                if(data["close"][i-p]-data[self.kijunsen][i-p]<0):
                    if(((data["close"][i-p-1:i-p]-data[self.kijunsen][i-p-1:i-p])).max()<0):
                        baseline_signal = 1
                    break
        
        #Base Line 2
        if(data[self.ema_close_1][i]-data[self.ema_close_2][i]>0):
            for p in range(1,7):
                if(data[self.ema_close_1][i-p]-data[self.ema_close_2][i-p]<0):
                    if(((data[self.ema_close_1][i-p-1:i-p]-data[self.ema_close_2][i-p-1:i-p])).max()<0):
                        baseline_signal = 1
                    break        
            
        ########################################################
        #confirmation_signal_1
        confirmation_signal_1=0
        if(data[self.ha_open][i]<data[self.ha_close][i]
          and data[self.ha_low][i]==data[self.ha_open][i]):
            confirmation_signal_1 = 1
        
        if(confirmation_signal_1==0):
            if(data[self.ha_open][i-1]<data[self.ha_close][i-1]
              and data[self.ha_low][i-1]==data[self.ha_open][i-1]
              and data[self.ha_open][i]<data[self.ha_close][i] and data[self.macd][i]>0 ):
                atr = data[self.atr][i-1]*1
                if(data["close"][i-1]+atr>data["close"][i]):
                    confirmation_signal_1 = 1
        
        ########################################################
        #confirmation_signal_2
        confirmation_signal_2=0
        if(data[self.macd][i]-data[self.macd_signal][i]>0):
            for p in range(1,5):
                if(data[self.macd][i-p]-data[self.macd_signal][i-p]<0):
                    if(((data[self.macd][i-p-1:i-p]-data[self.macd_signal][i-p-1:i-p])).max()<0):
                        atr = data[self.atr][i-p+1]*1
                        if(data["close"][i-p+1]+atr>data["close"][i]):
                            confirmation_signal_2 = 1
                    break
                    
        ########################################################
        #volumn_signal
        volumn_signal=0
        if(data[self.atr_stoploss_1][i]<data["close"][i]):
            volumn_signal = 1
            
        ########################################################
        if(shares_can_buy*data["close"][i]>1000 and shares_can_sell==0):
            if(i>20 and
                baseline_signal == 1
                and confirmation_signal_1 == 1
                and confirmation_signal_2 == 1
                and volumn_signal == 1
            ):
                if(shares_can_sell==0):
                    #setup ATR trailing stoploss 
                    data = self.set_stoploss(data, i , 14, risk_atr_multiplier, 
                                             early_stop_profit=target ,stop_early_times=1)
                action="BUY", data["close"][i], data
            
        if(shares_can_sell>0):
            if(i>20
                and data["stoploss"][i]>data["low"][i]
                and shares_can_sell > 0
            ):
                action = "STOPPED", data["stoploss"][i], data
            
        if(shares_can_sell>0
           and False
        ):
                action = "SELL", data["close"][i], data
            
        return action
    
    def get_col_to_plot(self):
        plot1 = []
        plot2 = []
        plot3 = []
        
        self.kijunsen = name_col("kijunsen",self.kijunsen_param_1)
        self.ema_close_1 = name_col("ema",self.ema_close_1_param_1,self.ema_close_1_param_2)
        self.ema_close_2 = name_col("ema",self.ema_close_2_param_1,self.ema_close_2_param_2)
        self.ha_open = "ha_open"
        self.ha_close = "ha_close"
        self.ha_high = "ha_high"
        self.ha_low ="ha_low"
        self.atr = name_col("atr",self.atr_param_1)
        self.macd = name_col("macd",self.macd_param_1,self.macd_param_2,self.macd_param_3)
        self.macd_signal = name_col("macd_signal",self.macd_param_1,self.macd_param_2,self.macd_param_3)
        self.macd_hist = name_col("macd_hist",self.macd_param_1,self.macd_param_2,self.macd_param_3)
        self.atr_stoploss_1 = name_col("atr_stoploss",self.atr_stoploss_1_param_1,
                              self.atr_stoploss_1_param_2,self.atr_stoploss_1_param_3,
                              self.atr_stoploss_1_param_4)
        plot1.append(self.kijunsen)
        plot1.append(self.ema_close_1)
        plot1.append(self.ema_close_2)
        plot1.append(self.ha_open)
        plot1.append(self.ha_close)
        plot1.append(self.ha_high)
        plot1.append(self.ha_low)
        plot1.append(self.atr_stoploss_1)
        
        plot2.append(self.macd)
        plot2.append(self.macd_signal)
        plot2.append(self.macd_hist)
        
        plot3.append(self.atr)
        
        return plot1,plot2,plot3
    

In [879]:
%%time
stock = "SGD=X"
stocks=get_forex()["stock"]
stocks=[stock]
sdate=date(2000, 1, 1)
edate=date(2019, 12, 1)
strategy=Strategy3()
stocks_prices={}
for stock in stocks:
    prices = get_data(stock,str(sdate),str(edate))
    result = back_test(prices, 
                                   cash = 100000,
                                   transaction_fee =0.00,
                                   assets={},
                                   strategy=Strategy3(), 
                                   target=0.1, 
                                   risk_percent = 0.01,
                                   risk_atr_multiplier =3)
print(str(result[0])+"%")


@@@
@@@


KeyboardInterrupt: 

In [863]:
plot_chart(stocks_prices, result, stock, strategy)

profit: -6.83%	asset: 93173.73334883302
	win count: 2	loss count: 3
	win percent: 0.01889735782036446	loss percent: -0.01907730000985227


FigureWidget({
    'data': [{'name': 'close',
              'type': 'scattergl',
              'uid': 'd8c6fc1…

In [757]:
%%time
def multi_backtest(stocks, total_tries, strategy=[StrategyMACD()], cash = 10000, transaction_fee =0.01, report='report', start = "2017-01-01",end = "2019-12-01"):
    df = pd.DataFrame(columns=['strategy', 'profit', 'start', 'end']) 
    
    for strgy in strategy:
        
        accumulated_growth = 0
        tries = 0
        for i in range(0,total_tries):
            for stock in stocks:
                strategy=StrategyMACD()
                stocks_prices={}
                prices = get_data(stock,str(start),str(end))
                stocks_prices[stock]=prices
                
                
                result = back_test([stock],stocks_prices,
                              sdate,
                              edate,
                              cash = 100000,
                              transaction_fee = 0.01,
                              assets={},
                              target=0.02,
                              risk_percent = 0.01,
                              risk_atr_multiplier = 3,
                              strategy=strategy)
                growth, longs, shorts,  win_trade_count, loss_trade_count, win_percent, loss_percent, final_cash=result
                print(stock+" # "+strgy.__class__.__name__)
                print("profit: "+str(round(growth,2))+"%"+"\t"+"asset: "+str(final_cash))
                print("\t"+"win count: "+str(win_trade_count[stock])+"\t"+"loss count: "+str(loss_trade_count[stock]))
                print("\t"+"win percent: "+str(win_percent[stock])+"\t"+"loss percent: "+str(loss_percent[stock]))
                profit = result[0]
                accumulated_growth = accumulated_growth + profit
                if(result[0]!=0):
                    tries = tries+ 1
        print("########## "+str(round(accumulated_growth/tries,2))+"% ######## tries: "+str(tries))
        df = df.append( {
            "strategy": strgy.__class__.__name__,
            "profit": round(accumulated_growth/tries,2),
            "start": start,
            "end": end
        }, ignore_index=True)
    df.to_csv(report, index = None, header=True)
    
strategyBatch1 = [StrategyMACD()]

strgy_batch= [strategyBatch1]
strgy_reportname= ["StrategyMACD"]
strgy_test=[1]
stocks  = get_forex()[0:100]["stock"].values
#stocks  = ["GOOGL","AAPL"]
#stocks  = get_stocks()[60:100]["stock"].values
#stocks  = get_sg_stocks()[0:10]["stock"].values

start = "2019-01-01"
end = "2019-12-01" 
for i in range(0,len(strgy_batch)):
    if(strgy_test[i]==1):
        reportname = strgy_reportname[i]
        multi_backtest(stocks, 1, strategy=strgy_batch[i], cash = 10000,
                       transaction_fee =0.000, report ="./report/" + reportname+ ".csv", start = start,end = end )

SGD=X # StrategyMACD
profit: -12.13%	asset: 87865.00249050002
	win count: 2	loss count: 5
	win percent: 0.025724009009701107	loss percent: -0.013693823410964086
EURUSD=X # StrategyMACD
profit: -14.78%	asset: 85224.97444902999
	win count: 1	loss count: 9
	win percent: 0.007973349352848108	loss percent: -0.06460189493052144
JPY=X # StrategyMACD
profit: -16.53%	asset: 83468.81841999994
	win count: 1	loss count: 11
	win percent: 0.016173168016003308	loss percent: -0.07401256059474663
GBPUSD=X # StrategyMACD
profit: -8.35%	asset: 91651.59323384005
	win count: 2	loss count: 7
	win percent: 0.03565962593408208	loss percent: -0.050852715984405855
AUDUSD=X # StrategyMACD
profit: -6.24%	asset: 93755.34403811999
	win count: 2	loss count: 5
	win percent: 0.011592359189634062	loss percent: -0.022701207229972226


KeyboardInterrupt: 

In [ ]:
def back_test(stock,stocks_prices,
              sdate,
              edate,
              cash = 10000000,
              transaction_fee = 0.01,
              assets={},
              target=0.02,
              risk_percent = 0.01,
              risk_atr_multiplier = 3,
              strategy=StrategyMACD()):
    init_cash = cash
    
    longs = {}
    shorts = {}
    bought_at = {}
    bought_at_list = {}
    loss_trade_count = {}
    win_trade_count = {}
    win_percent = {}
    loss_percent = {}
    win_amount = {}
    loss_amount = {}
    total_stock = len(stocks)
    from datetime import date, timedelta
    if(total_stock==1):
        loop = stocks_prices[stocks[0]].iterrows()
    else:
        loop = daterange(sdate, edate)
    for index in loop:
        if(total_stock==1):
            i,row = index
            day = str(row.date)
        else:
            i=[]
            day = str(index)
        for stock in stocks:
            if(len(stocks)>1):
                i = stocks_prices[stock][stocks_prices[stock]["date"]==day].index.values
                if(len(i)==0):
                    continue
                else:
                    i=i[0]
                row = stocks_prices[stock].iloc[i]
            data = stocks_prices[stock]
            shares_can_buy = 0
            shares_can_sell=0
            if stock in assets:
                shares_can_sell = assets[stock]
            
            # Risk management
            shares_can_buy = risk_management(total_stock,i,row,risk_atr_multiplier,risk_percent,day,stock,stocks_prices,cash,assets,transaction_fee)
   
            # Get action
            action , price_point, data = strategy.get_action(stocks_prices[stock], i, shares_can_buy, shares_can_sell, target, risk_percent, risk_atr_multiplier)
            
            # Execution
            if(stock not in longs):
                longs[stock]={}
            if(stock not in shorts):
                shorts[stock]={}
            if(stock not in bought_at_list):
                bought_at_list[stock]=[]
            if(stock not in bought_at):
                bought_at[stock]=100000
            if(stock not in loss_trade_count):
                loss_trade_count[stock]=0
            if(stock not in win_trade_count):
                win_trade_count[stock]=0
            if(stock not in loss_percent):
                loss_percent[stock]=0
            if(stock not in win_percent):
                win_percent[stock]=0
            if(stock not in loss_amount):
                loss_amount[stock]=0
            if(stock not in win_amount):
                win_amount[stock]=0
                
            if(action=="BUY"):
                if(shares_can_buy>0):
                    cost = shares_can_buy*price_point*(1+transaction_fee)
                    cash_left = cash-cost
                    if stock not in assets:
                        assets[stock]=0
                    assets[stock] = assets[stock] + shares_can_buy
                    cash = cash_left
                    longs[stock][row.date] = price_point
                    bought_at[stock] = price_point
                    bought_at_list[stock] = bought_at_list[stock] + [price_point]
                    #print(stock+" BUY : +" +str(shares_can_buy) +" @"+str(price_point))
            if(action=="SELL"):
                if(shares_can_sell>0):
                    asset = shares_can_sell * price_point
                    cash = cash + asset*(1-transaction_fee)
                    assets[stock]=0
                    shorts[stock][row.date] = price_point
                    #trade counter
                    for trade in bought_at_list[stock]:
                        if(trade>price_point):
                            loss_trade_count[stock] = loss_trade_count[stock] +1
                            loss_percent[stock] = loss_percent[stock] + ((price_point-trade)/trade)
                        else:
                            win_trade_count[stock] = win_trade_count[stock] +1
                            win_percent[stock] = win_percent[stock] + ((price_point-trade)/trade)
                    bought_at_list[stock] = []
                    #print(stock+" SELL : -" +str(shares_can_sell) +" @"+str(price_point))
            if(action=="STOPPED"):
                if(shares_can_sell>0):
                    asset = shares_can_sell * price_point
                    cash = cash + asset*(1-transaction_fee)
                    assets[stock]=0
                    shorts[row.date] = price_point
                    #trade counter
                    for trade in bought_at_list[stock]:
                        if(trade>price_point):
                            loss_trade_count[stock] = loss_trade_count[stock] +1
                            loss_percent[stock] = loss_percent[stock] + ((price_point-trade)/trade)
                        else:
                            win_trade_count[stock] = win_trade_count[stock] +1
                            win_percent[stock] = win_percent[stock] + ((price_point-trade)/trade)
                    bought_at_list[stock] = []
                    #print(stock+" STOPPED : -" +str(shares_can_sell) +" @"+str(price_point))
            #print (get_current_asset(day, stocks_prices, cash,  assets) )       
    # Compute result
    final_cash = cash
    for holding_stock in assets:
        
        holding_data = stocks_prices[holding_stock]
        holding_shares_asset = assets[holding_stock]*holding_data.iloc[-1].close
        final_cash = final_cash+holding_shares_asset
    assets.clear()
    growth =((final_cash-init_cash)/init_cash)*100
    return round(growth,2), longs, shorts,  win_trade_count, loss_trade_count, win_percent, loss_percent, final_cash

In [ ]:
def get_current_asset(total_stock, i, row, day, stocks_prices, cash,  assets):
    current_assets = cash
    for holding_stock in assets:
        holding_data = stocks_prices[holding_stock]
        if(total_stock>1):
            pos = holding_data[holding_data["date"]==day].index.values
            if(len(pos)>0):
                pos = pos[0]
        else:
            pos = i
        holding_shares_asset = assets[holding_stock]*holding_data.iloc[pos-1].close
        current_assets = current_assets+holding_shares_asset
    return current_assets

def risk_management(total_stock,i,row,risk_atr_multiplier,risk_percent,day,stock,stocks_prices,cash,assets,transaction_fee):
    data = stocks_prices[stock]
    if(total_stock>1):
        i = data[data["date"]==day].index.values
        row = data.iloc[i]
    if(i>0):
        shares_can_buy = int(cash/(row.close*(1+transaction_fee)))
        shares_can_sell=0
        if stock in assets:
            shares_can_sell = assets[stock]
            recommended_shares = 0
        
        #compute current asset
        current_assets = get_current_asset(total_stock, i, row, day, stocks_prices, cash, assets)
        
        #compute
        process_col(data,"atr_risk",14)
        risk_atr = data["atr_risk_14"][i-1]
        last_close = data["close"][i-1]
        
        if(shares_can_buy>0):
            import math
            if(math.isnan(risk_atr)==False):
                risk_atr =  risk_atr * risk_atr_multiplier
                risk_amt = current_assets * risk_percent
                if(risk_atr!=0):
                    recommended_shares = int(risk_amt/risk_atr)
                    if(shares_can_sell+shares_can_buy>recommended_shares):
                        shares_can_buy = int(recommended_shares-shares_can_sell) 
                        return shares_can_buy
                    else:
                        return shares_can_buy
                else:
                    return 0
            else:
                return 0        
        else:
            return 0
        
    else:
        return 0
    